In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
import mne
import pathlib
import sklearn
import seaborn as sns
from meegkit import dss
 
from pycrostates.preprocessing import extract_gfp_peaks
from neurokit2.microstates.microstates_clean import microstates_clean
from mne_icalabel import label_components
from scipy.stats import kurtosis, skew

In [2]:
from eeg_clean import epoch_stats, channel_stats, clean

In [3]:
def zapline_clean(raw, fline):
    data = raw.get_data().T # Convert mne data to numpy darray
    sfreq = raw.info['sfreq'] # Extract the sampling freq
   
    #Apply MEEGkit toolbox function
    out, _ = dss.dss_line(data, fline, sfreq, nkeep=1) # fline (Line noise freq) = 50 Hz for Europe
    print(out.shape)

    cleaned_raw = mne.io.RawArray(out.T, raw.info) # Convert output to mne RawArray again

    return cleaned_raw

In [20]:
data_folder = pathlib.Path(r"C:\Users\Gulbr\MasterOppgave\Data\epilepsy_data")
file = data_folder / "TLE_0011.bdf"


raw = mne.io.read_raw_bdf(file, verbose=False)
print(raw.info)
raw.crop(0, 1050).load_data()
raw.drop_channels(['SO2', 'IO2', 'LO1', 'LO2', 'EXG5', 'EXG6', 'EXG7', 'EXG8'])
noe = np.where(raw.get_data()[-1, :] > 0.1)
#raw.drop_channels("Status")
#raw.set_montage('biosemi128')

#raw_highpass = raw.copy().filter(l_freq=1, h_freq=None, verbose=False)
#raw_lowpass = raw_highpass.copy().filter(l_freq=None, h_freq=100, verbose=False)
#line_noise = zapline_clean(raw_lowpass, 50)
#raw_down_sampled = line_noise.copy().resample(sfreq=200, verbose=False)

<Info | 8 non-empty values
 bads: []
 ch_names: A1, A2, A3, A4, A5, A6, A7, A8, A9, A10, A11, A12, A13, A14, ...
 chs: 136 EEG, 1 Stimulus
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 417.0 Hz
 meas_date: 2018-09-25 14:06:03 UTC
 nchan: 137
 projs: []
 sfreq: 2048.0 Hz
 subject_info: 1 item (dict)
>
Reading 0 ... 2150400  =      0.000 ...  1050.000 secs...


In [22]:
for i in noe[0]:
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

Bad epoch idenfication - based on topographic stability

Between GFP peaks

In [ ]:
ones_epochs = mne.make_fixed_length_epochs(raw_down_sampled, 0.5, verbose=False, preload=True)

In [ ]:
stats = epoch_stats.EpochStats(ones_epochs, switching_frq_thresholds=np.array([1, 1.4]))
stats.calc_stability()

In [ ]:
epoched_stability = stats.quasi_stability.get_mean_abs_stab()
convex_stabilty = stats.quasi_stability.get_mean_stab()
epoched_switching_freq = stats.quasi_stability.switching_freqs[:, 0]
significant_switching_freq = stats.quasi_stability.switching_freqs[:, 1]
peaks = stats.n_gfp_peaks

Plots to investigate exclusion of epocs not following our assumptions.

Epoch averaged DISS

In [ ]:
plt.hist(convex_stabilty, bins=200)
plt.show()

Stability transformed: absoluteDISS = |DISS-1| ... and averaged for each epoch. Now only high values indicate stability.

In [ ]:
plt.hist(epoched_stability, bins=200)
plt.show()

Average epoch polarity inversion going from gfp peak to gfp peak, meaning DISS < 1.

In [ ]:
plt.hist(epoched_switching_freq, bins=36)
plt.show()

The same again, but only for shifts in polarity where higher stability is indicated (DISS > 1.4)

In [ ]:
plt.hist(significant_switching_freq, bins=21)
plt.show()

Number of GFP peaks for each epoch

In [ ]:
plt.hist(peaks, bins=21)
plt.show()

Looking at scatter plot between swithcing frequencies and number of GFP peaks

In [ ]:
plt.scatter(peaks, epoched_switching_freq)
plt.show()

In [ ]:
plt.scatter(peaks, significant_switching_freq)
plt.show()

Looking at average absoluteDISS in relation to switching frequencies.

In [ ]:
plt.scatter(epoched_stability, epoched_switching_freq)
plt.show()

In [ ]:
plt.scatter(epoched_stability, significant_switching_freq)
plt.show()

Same but with average DISS

In [ ]:
plt.scatter(convex_stabilty, epoched_switching_freq)
plt.show()

In [ ]:
plt.scatter(convex_stabilty, significant_switching_freq)
plt.show()

Scatter of the two stability types

In [ ]:
plt.scatter(convex_stabilty, epoched_stability)
plt.show()

In [ ]:
fit = np.poly1d([1, 1])
polyline = np.linspace(-1.25, 0, 100)
reg = np.poly1d([0.8, 1.2])
reg2 = np.poly1d([0.5, 1.2])
plt.scatter(convex_stabilty-2, epoched_stability)
plt.plot(polyline, fit(polyline), color='red')
plt.plot(polyline, reg(polyline), color="blue")
plt.plot(polyline, reg2(polyline), color="black")
plt.show()


In [ ]:
plt.scatter(epoched_switching_freq, significant_switching_freq)
plt.show()

At GFP peaks

In [ ]:
at_epoched_stability = stats.peak_stability.get_mean_abs_stab()
at_convex_stabilty = stats.peak_stability.get_mean_stab()

In [ ]:
plt.hist(at_epoched_stability, bins=200)
plt.show()

In [ ]:
plt.hist(at_convex_stabilty, bins=200)
plt.show()

In [ ]:
plt.scatter(at_convex_stabilty, at_epoched_stability)
plt.show()

In [ ]:
fit = np.poly1d([-0.9, 1])
reg = np.poly1d([-1, 1])
reg2 = np.poly1d([-0.7, 1])
polyline = np.linspace(0, 2, 100)
plt.scatter(at_convex_stabilty, at_epoched_stability)
plt.plot(polyline, fit(polyline), color='red')
plt.plot(polyline, reg(polyline), color="blue")
plt.plot(polyline, reg2(polyline), color="black")
plt.show()

In [ ]:
print(fit)

In [ ]:
def scale(values):
    v = values.reshape(-1, 1)
    scaler = sklearn.preprocessing.StandardScaler()
    noe = scaler.fit_transform(v)
    return noe

Bad channel evaluation

In [ ]:
stats_channels = channel_stats.ChannelStats(ones_epochs, sklearn_scaler=sklearn.preprocessing.MinMaxScaler())

Channel Satbility

At GFP peaks

In [ ]:
at_test1 = stats_channels.peak_stab_change
test1 = stats_channels.quasi_stab_change

In [ ]:
plt.hist(at_test1*128, bins=64)
plt.show()

In [ ]:
order = np.argsort(at_test1)
n = 126
print(skew(at_test1[order][:n]))
print(kurtosis(at_test1[order][:n]))
print(np.array(stats_channels.ch_name_list)[order])

Between GFP peaks

In [ ]:
plt.hist(test1*128, bins=64)
plt.show()

In [ ]:
order = np.argsort(test1)
n = 121
print(skew(test1[order][:n]))
print(kurtosis(test1[order][:n]))
print(np.array(stats_channels.ch_name_list)[order])

In [ ]:
print(kurtosis(test1))
print(skew(test1))

test threshold exclusions..

In [ ]:
clean_test = clean.Clean(ones_epochs)

In [ ]:
bad_epochs = clean_test.find_bad_epochs(peaks_args={"method": "threshold", "threshold": 0.8, "exclude": "bigger"})

In [ ]:
print(bad_epochs)

In [ ]:
bad_epochs = clean_test.find_bad_epochs(peaks_args={"method": "function_threshold", "function": np.poly1d([-1, 1.2]), "exclude":"bigger"})

In [ ]:
print(bad_epochs)

PCA heuristic idea

In [ ]:
pc_all_chanels = stats_channels.pca_auc_change

In [ ]:
plt.hist(scale(pc_all_chanels), bins=64)
plt.show()

In [ ]:
order = np.argsort(pc_all_chanels)
n = 128
print(skew(pc_all_chanels[order][:n]))
print(kurtosis(pc_all_chanels[order][:n]))
print(stats_channels.ch_name_list[order])

In [ ]:
bad_by_corr = np.where(pc_all_chanels > 0)[0]
print(stats_channels.ch_name_list[bad_by_corr])

In [ ]:
idx = np.where((at_test1 > 0) & (test1 > 0) & (pc_all_chanels > 0))[0]
print(idx)

In [ ]:
print(stats_channels.ch_name_list[idx])

In [ ]:
idx = np.where((at_test1 > 0) & (pc_all_chanels > 0))[0]
idx2 = np.where((test1 > 0) & (pc_all_chanels > 0))[0]
print(idx)
print(idx2)

In [ ]:
print(stats_channels.ch_name_list[idx])
print(stats_channels.ch_name_list[idx2])

Channels to exclude

In [ ]:
acum = []
for cn in stats_channels.ch_name_list[np.argsort(test1)]:
    if cn in stats_channels.ch_name_list[bad_by_corr]:
        print(cn)
        acum.append(cn)
    else:
        break
for cn in stats_channels.ch_name_list[np.argsort(at_test1)]:
    if cn in stats_channels.ch_name_list[bad_by_corr]:
        if cn not in acum:
            print(cn)
    else:
        break


Evaluate

In [ ]:
from data_quality import ica_score

In [ ]:
avref_epochs = raw_down_sampled.copy().set_eeg_reference()
evaluate = ica_score.IcaScore(avref_epochs)

In [ ]:
evaluate.ica.plot_components()

In [ ]:
print(evaluate.ica_labels)